# Alabama Crash Data 2018 – Data Quality and Cleaning

This notebook covers **Task - 1: Data Quality and Cleaning**. We load, tidy, and assess data quality for the 2018 Alabama crash dataset (22 selected variables).

## Goals:
- Verify that each row represents a single crash event.
- Ensure variables have appropriate data types.
- Treatment missing and unknown values.
- Transformation of multiple categorical options to fewer standard outcomes
- Create a crash‐severity variable that will be used in later EDA.

## 2.1. Setup and loading the dataset
The original dataset contained 160,163 crash records with 235 columns. Many of these columns included administrative information or repeated data. To keep the scope of the present study focused, we selected only the variables related to crash severity, crash occurrence, roadway characteristics, environmental conditions, driver demographics, and DUI-related factors. In total, 22 relevant variables were chosen. A separate CSV file containing these 22 columns was created and loaded for analysis.

In [ ]:
import pandas as pd
import numpy as np

# Display options
pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)

# Load dataset
df = pd.read_csv("../data/raw/Raw_Data.csv")

### 2.1.1. Initial structure

We start by confirming the basic structure of the dataset and verifying that each row corresponds to one crash event.


In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.info()

### 2.1.2. Description of selected variables
Our 22 variables capture crash outcomes, roadway environment, traffic characteristics, and key driver factors. The injury count variables (`Number Killed`, `Number Serious Injuries`, `Number Non-fatal Injuries`) quantify crash outcomes and will be combined into an overall crash severity measure. Temporal and location context is provided by `DateTime`, `County`, `Area Type` (rural/urban), and `Functional Class` (e.g., local, collector, arterial, interstate).

Crash circumstances are described by `Crash Manner` (e.g., rear-end, angle, single-vehicle), `Visibility Obstruction`, `Lighting Conditions`, `Roadway Curvature and Grade`, `Lane Separation`, `Number of Lanes`, and `AADT (Average Annual Daily Traffic)` as a traffic-exposure proxy. Traffic units and driver characteristics are captured through `Number of Vehicles`, `Vehicle Type`, `Driver Gender`, `Driver License Status`, `Driver Age`, `Driver BAC`, `Impact Speed`, and `Speed Limit`, which together allow exploration of behavioral and speed-related factors associated with more severe outcomes.

In [ ]:
df.describe()

### 2.1.3. Overall Observation
The full crash dataset contains 160,163 crash-level observations and 22 variables, which matches our planned list of roadway, environmental, traffic, and driver-related fields. Six variables are stored as numeric from the start (injury counts, AADT, Driver Age, and BAC), while the remaining 16 are objects that encode dates or categorical descriptors and will need type conversion before analysis. The descriptive statistics show that most crashes have zero recorded injuries, AADT values span from a special negative code (−1) up to about 151,000, and driver age ranges from 0 to 150, indicating both plausible variation and a few values that warrant closer checking (e.g., very young/old ages and negative AADT).

## 2.2. Type conversion and basic tidying
Next, we convert key columns to appropriate types and derive a few helper variables (time fields, severity index, cleaned speeds). We created a python helper file for data cleaning named `cleaning.py` to meet following objectives:

1. To derive hour, weekday, and a weekend flag from `DateTime`, and create a simple `Time of Day` variable that classifies crashes as **Daytime** (06:00–17:59) or **Nighttime** (18:00–05:59). This will allow us to compare crash severity and conditions between day and night in later EDA.
2. To coerce injury counts, driver age, BAC, and AADT to numeric. Any non-numeric entries (e.g., blanks, text codes) become `NaN` so they can be handled explicitly.
3. To transform `Impact Speed` stored as text ranges (e.g., "11 to 15 MPH") or single values. We extract all numeric values in the string and, when a range is present, replace it with the midpoint (average of the two bounds); single values are kept as-is and unknown entries become missing in the numeric variable `Impact Speed Num`.
4. To transform `Number of Vehicles` stored as text labels (e.g., "2 Vehicles") to numeric (2.0). Extracted the count from `Number of Vehicles`
5. To transform `Speed Limit` stored as text labels (e.g., "35 MPH") to numeric (35.0). Removed the "MPH" suffix from `Speed Limit`
6. To transform `Number of Lanes` stored as text labels (e.g., "Six Lanes or More") to numeric (6.0). Mapped lane descriptions to lane counts, treating "Six Lanes or More" as 6 and parking-lot/unknown values as missing.

In [ ]:
import cleaning_utils as cu

df = pd.read_csv("../data/raw/Raw_Data.csv")

# Time variables
df = cu.add_time_variables(df)

# Coerce core numeric columns
numeric_cols = [
    "Number Killed",
    "Number Serious Injuries",
    "Number Non-fatal Injuries",
    "Driver Age",
    "Driver BAC",
    "AADT",
]
df = cu.coerce_numeric(df, numeric_cols)

# Derived numeric versions
df["Impact Speed Num"] = cu.impact_speed_to_numeric(df["Impact Speed"])
df["Number of Vehicles Num"] = cu.vehicles_to_numeric(df["Number of Vehicles"])
df["Speed Limit Num"] = cu.speed_limit_to_numeric(df["Speed Limit"])
df["Number of Lanes Num"] = cu.lanes_to_numeric(df["Number of Lanes"])

## 2.3. Creating crash severity variable
We combine the injury count fields into an ordinal `Crash Severity` variable for use as our main outcome in later analyses. We define four mutually exclusive categories:

- **PDO**: Property Damage Only: no recorded injuries.
- **Minor**: at least one non-fatal injury, but no serious injuries or fatalities.
- **Serious**: at least one serious injury, but no fatalities.
- **Fatal**: at least one fatality.

This ordered factor will anchor many of our subsequent EDA plots.

In [ ]:
df["Total Injuries"] = (
    df["Number Killed"] + df["Number Serious Injuries"] + df["Number Non-fatal Injuries"]
)


# FHWA crash severity matrix
def categorize_severity(row):
    if row["Number Killed"] > 0:
        return "Fatal"
    elif row["Number Serious Injuries"] > 0:
        return "Serious"
    elif row["Total Injuries"] > 0:
        return "Minor"
    else:
        return "PDO"


df["Crash Severity"] = df.apply(categorize_severity, axis=1)

severity_order = ["PDO", "Minor", "Serious", "Fatal"]
df["Crash Severity"] = pd.Categorical(df["Crash Severity"], categories=severity_order, ordered=True)

df["Crash Severity"].value_counts().sort_index()

## 2.4. Data quality assessment

We now examine missing data, special codes (e.g., -1, "Unknown"), and basic ranges for numeric variables.


### 2.4.1. Overall missingness

In [ ]:
missing_prop = df.isna().mean().sort_values(ascending=False)
missing_prop

### Observation
The table above shows the proportion of missing values in each variable. We will pay particular attention to any variables with substantial missingness (for example, driver BAC or impact speed). The missingness profile shows that most variables are fully observed, but a few have substantial gaps. `Driver BAC` is missing for about 99% of crashes, reflecting that BAC is only recorded when DUI is suspected or a test is performed, so this is structural rather than random missingness. `Impact Speed Num`, `AADT`, `Lane Separation`, `Driver Age`, `Speed Limit Num`, and `Number of Lanes Num` have moderate missingness, often due to text codes, unmeasured values, or special codes (e.g., negative AADT) that were recoded as `NaN` during cleaning. For subsequent analyses we keep all crashes, treat these missing values as “data not recorded,” and use BAC only in a focused subsample where it is available, rather than restricting the main dataset.

### 2.4.2 Fix negative/special numeric codes (e.g., AADT)

In [ ]:
# Get a list of numeric columns
df.columns
df.select_dtypes(include="number").columns

In [ ]:
# Check ranges for main numeric variables
df[
    [
        "Number Killed",
        "Number Serious Injuries",
        "Number Non-fatal Injuries",
        "Driver Age",
        "Driver BAC",
        "Impact Speed Num",
        "Speed Limit Num",
        "Number of Vehicles Num",
        "Number of Lanes Num",
        "Total Injuries",
        "AADT",
    ]
].describe()

### Observation
The extended numeric summary shows that crashes typically involve about 2 vehicles and 3–4 lanes, with very few cases exceeding 4 vehicles or 6 lanes. Injury counts remain zero for most crashes, but when injuries occur the total can reach up to 47 people in a single event, indicating a small number of high‑severity, multi‑party crashes. Driver age has a median of 34 years and an interquartile range of roughly 23–52 years, but spans from 0 to 150, suggesting a few implausible ages that may reflect data entry or coding issues. Posted speed limits mostly fall between 35 and 55 mph and impact speed midpoints cluster around 30 mph, while AADT again ranges from a special code (−1, treated as missing) to over 150,000, capturing both low‑volume rural roads and high‑volume urban facilities.

In [ ]:
# Recode negative AADT as missing
df.loc[df["AADT"] < 0, "AADT"] = np.nan

# Flag clearly implausible ages
implausible_ages = (df["Driver Age"] < 10) | (df["Driver Age"] > 100)
implausible_ages.sum()

### Observation
A small number of driver ages appear implausible: 66 records have ages below 10 or above 100 years. These values likely reflect data entry or coding errors rather than true driver characteristics, we set them to NaN.

In [ ]:
# Setting implausible ages to NaN
df.loc[implausible_ages, "Driver Age"] = np.nan

In [ ]:
# Creating a BAC availability flag
df["BAC Available"] = ~df["Driver BAC"].isna()
df["BAC Available"].mean()  # proportion tested

### Observation
Only about 1.2% of crashes have a recorded `Driver BAC`, confirming that BAC is documented primarily when DUI is suspected or a test is performed rather than for all drivers. We create a DUI subset.

In [ ]:
# DUI subset creation
dui_df = df[df["BAC Available"]].copy()

### 2.4.3. Recoding categorical variables

To simplify interpretation and align variables with our research questions, we recoded several detailed categorical fields into cleaner, analysis-ready versions. `Functional Class Recode` collapses the original FHWA classes by merging “Principal Arterial – Other” and “Principal Arterial – Other Freeways or Expressways” into a single **Principal Arterial** category, combines **Major Collector** and **Minor Collector** into **Collector**, and treats explicit “Null value” entries as missing. For crash type, `Crash Manner Recode` groups a variety of side-impact, angle, and backing side collisions into **Sideswipe / Angle**, combines frontal oncoming configurations into **Head-On**, merges rear-end and backing rear-to-rear crashes into **Rear-End**, sets “Unknown” to missing, and classifies all remaining patterns as **Others**.

Environmental and roadway context variables were also streamlined. `Visibility Obstruction Recode` distinguishes three states: **No** when not obscured, **Yes** for any specific obstruction, and missing when the unit is unknown. `Lighting Conditions Recode` aggregates many detailed lighting codes into five interpretable categories: **Daylight**, **Illuminated** (dark with spot illumination), **Night-lighted** (dark with continuous street lighting), **Dark** (roadway not lighted), and **Dusk/Dawn**, while unknown or not applicable values are set to missing and rare patterns labeled **Others**. For cross-section design, `Raised Median` is a binary indicator derived from `Lane Separation` that takes **Yes** when a physical separator (paved surface, concrete barrier, metal guardrail, or unpaved surface) is present, **No** for other markings, and missing when separation is not applicable or unknown. In addition, `Curvature` and `Grade` flags are extracted from `Roadway Curvature and Grade`, indicating whether a crash occurred on any curve (**Yes/No**) and whether vertical grade features (up/down grade, hillcrest, sag) were present.

Vehicle and driver characteristics were recoded to focus on meaningful categories and reduce unknowns. `Vehicle Type Recode` retains major light-vehicle and truck types (passenger cars, SUVs, pickups, minivans, selected trucks, cargo vans, motorcycles), treats explicitly unknown/unspecified motorized vehicles as missing, and collapses all remaining rare types into **Others**. `Driver Gender Recode` keeps only **Male** and **Female**, setting all other statuses to missing. Finally, `Driver License Validity` summarizes licensing into a simple indicator: **Yes** for current/valid licenses, **No** for any clearly invalid or non-current status (e.g., suspended, revoked, expired), and missing when the status is unknown or the record does not correspond to a vehicle driver. Together, these recodes reduce sparse categories and ambiguous codes, making the dataset more interpretable


In [ ]:
# Get a list of columns
df.columns

In [ ]:
# Select categorical columns
cat_cols = [
    "County",
    "Area Type",
    "Functional Class",
    "Crash Manner",
    "Visibility Obstruction",
    "Lighting Conditions",
    "Vehicle Type",
    "Driver Gender",
    "Driver License Status",
    "Roadway Curvature and Grade",
    "Lane Separation",
    "Weekend",
    "Time of Day",
]

# Investigate categorical options
for col in cat_cols:
    print(f"\n----- {col} -----")
    print(df[col].value_counts(dropna=False).head(30))

In [ ]:
# RECODING
# -----------------------------
# Functional Class recode (5 class by FHWA)
# -----------------------------
df["Functional Class Recode"] = df["Functional Class"]

df["Functional Class Recode"] = df["Functional Class Recode"].replace(
    {
        "Principal Arterial - Other": "Principal Arterial",
        "Principal Arterial - Other Freeways or Expressways": "Principal Arterial",
        "Major Collector": "Collector",
        "Minor Collector": "Collector",
        "Null value": np.nan,
    }
)

# -----------------------------
# Crash Manner recode
# -----------------------------
sideswipe_set = {
    "Sideswipe - Same Direction",
    "Sideswipe - Opposite Direction",
    "Side Impact (90 degrees)",
    "Side Impact (angled)",
    "Angle (front to side) Opposite Direction",
    "Angle (front to side) Same Direction",
    "Causal Veh Backing: Rear to Side",
}

headon_set = {"Angle Oncoming (frontal)", "Head-On (front to front only)"}

rear_set = {"Rear End (front to rear)", "Causal Veh Backing: Rear to Rear"}

unknown_set = {"Unknown"}


def recode_crash_manner(x):
    if x in sideswipe_set:
        return "Sideswipe / Angle"
    elif x in headon_set:
        return "Head-On"
    elif x in rear_set:
        return "Rear-End"
    elif x in unknown_set:
        return np.nan
    else:
        return "Others"


df["Crash Manner Recode"] = df["Crash Manner"].apply(recode_crash_manner)


# -----------------------------
# Visibility Obstruction recode
# -----------------------------
def recode_visibility(x):
    if x == "CU is Unknown":
        return np.nan
    elif x == "Not Obscured":
        return "No"
    else:
        return "Yes"


df["Visibility Obstruction Recode"] = df["Visibility Obstruction"].apply(recode_visibility)

# -----------------------------
# Lighting Conditions recode
# -----------------------------
illuminated = {
    "E Dark - Spot Illumination Both Sides of Roadway",
    "E Dark - Spot Illumination One Side of Roadway",
}

night_lighted = {
    "E Dark - Continuous Lighting Both Sides of Roadway",
    "E Dark - Continuous Lighting One Side of Roadway",
    "Dark - Roadway Lighted",
}


def recode_lighting(x):
    if x in illuminated:
        return "Illuminated"
    elif x in night_lighted:
        return "Night-lighted"
    elif x == "Dark - Roadway Not Lighted":
        return "Dark"
    elif x in ["Dusk", "Dawn"]:
        return "Dusk/Dawn"
    elif x in ["Unknown", "E Dark - Unknown Roadway Lighting", "Not Applicable"]:
        return np.nan
    elif x == "Daylight":
        return "Daylight"
    else:
        return "Others"


df["Lighting Conditions Recode"] = df["Lighting Conditions"].apply(recode_lighting)

# -----------------------------
# Vehicle Type recode
# -----------------------------
vehicle_unknown = {"E Unknown Type of Motorized Vehicle", "CU is Unknown", "Unknown"}

vehicles = {"Passenger Car", "Motorcycle"}

pickup = {"Pick-Up (Four-Tire Light Truck)"}

suv = {"E Sport Utility Vehicle (SUV)", "E 4-Wheel Off Road ATV"}

trucks = {
    "E Single-Unit Truck (2-Axle/6-Tire)",
    "E Tractor/Semi-Trailer",
    "E Single-Unit Truck (3 Axles or Less)",
    "E Truck (6 or 7) with Trailer",
    "E Other Heavy Truck (Cannot Classify)",
    "E Truck Tractor Only (Bobtail)",
    "E Tractor/Doubles",
    "E Low Speed Vehicle",
    "E Other Light Truck (10000 lbs or Less)",
}

vans = {"E Mini-van", "E Cargo Van (10000 lbs or Less)", "E Passenger Van", "E Van or Mini-Van"}


def recode_vehicle_type(x):
    if x in vehicle_unknown:
        return np.nan
    elif x in vehicles:
        return x
    elif x in pickup:
        return "Pickup"
    elif x in vans:
        return "Van"
    elif x in trucks:
        return "Truck"
    else:
        return "Other"


df["Vehicle Type Recode"] = df["Vehicle Type"].apply(recode_vehicle_type)


# -----------------------------
# Driver Gender recode
# -----------------------------
def recode_gender(x):
    if x in ["Male", "Female"]:
        return x
    else:
        return np.nan


df["Driver Gender Recode"] = df["Driver Gender"].apply(recode_gender)


# -----------------------------
# Driver License Validity
# -----------------------------
def recode_license_validity(x):
    if x == "Current/Valid":
        return "Yes"
    elif x in ["Unknown", "CU is Not a Vehicle"]:
        return np.nan
    else:
        return "No"


df["Driver License Validity"] = df["Driver License Status"].apply(recode_license_validity)

# -----------------------------
# Raised Median indicator from Lane Separation
# -----------------------------
raised_yes = {"Paved Surface", "Concrete Barrier", "Metal Guard Rail", "Unpaved Surface"}


def recode_raised_median(x):
    if x in ["NaN", np.nan, "Not Applicable", "CU is Unknown"]:
        return np.nan
    elif x in raised_yes:
        return "Yes"
    else:
        return "No"


df["Raised Median"] = df["Lane Separation"].apply(recode_raised_median)

# -----------------------------
# Curvature and Grade separation
# -----------------------------
curved_keywords = ["Curve"]
grade_keywords = ["Down Grade", "Up Grade", "Hillcrest", "Sag"]


def has_curvature(x):
    if pd.isna(x):
        return np.nan
    text = str(x)
    # any curve (left/right) counts as Yes
    return "Yes" if "Curve" in text else "No"


def has_grade(x):
    if pd.isna(x):
        return np.nan
    text = str(x)
    return "Yes" if any(k in text for k in grade_keywords) else "No"


df["Curvature"] = df["Roadway Curvature and Grade"].apply(has_curvature)
df["Grade"] = df["Roadway Curvature and Grade"].apply(has_grade)

# Make them categorical
df["Curvature"] = df["Curvature"].astype("category")
df["Grade"] = df["Grade"].astype("category")

## 2.5. Summarizing and saving cleaned dataset
### 2.5.1. Creating cleaned datset
We now create a cleaned datset including the recoded and modified columns of interest.

In [ ]:
# Get a list of columns
df.columns

In [ ]:
# Select the columns to keep
keep_cols = [
    # Severity Related
    "Number Killed",
    "Number Serious Injuries",
    "Number Non-fatal Injuries",
    "Total Injuries",
    "Crash Severity",
    # time/location
    "Weekend",
    "Time of Day",
    "County",
    "Area Type",
    # roadway / traffic
    "Functional Class Recode",
    "AADT",
    "Curvature",
    "Grade",
    "Raised Median",
    "Number of Lanes Num",
    "Number of Vehicles Num",
    "Speed Limit Num",
    "Impact Speed Num",
    # driver / vehicle
    "Vehicle Type Recode",
    "Driver Gender Recode",
    "Driver License Validity",
    "Driver Age",
    "Driver BAC",
    "BAC Available",
    # environment
    "Crash Manner Recode",
    "Visibility Obstruction Recode",
    "Lighting Conditions Recode",
]

# Create a cleaned dataframe
cleaned_df = df[keep_cols].copy()

# Convert object to category
obj_cols = cleaned_df.select_dtypes(include="object").columns
for col in obj_cols:
    cleaned_df[col] = cleaned_df[col].astype("category")

# Check
cleaned_df.info()

### 2.5.2. Saving cleaned datset
We save both the full working dataframe and the cleaned analysis dataset in pickle format (`crash_2018_full.pkl` and `crash_2018_cleaned.pkl`). These binary files preserve dtypes (including categorical variables) and can be loaded quickly in later notebooks for univariate, bivariate, and multivariate EDA.

In [ ]:
# Save full working dataframe (with originals + recodes)
df.to_pickle("../data/cleaned/crash_2018_full.pkl")

# Save cleaned analysis dataset
cleaned_df.to_pickle("../data/cleaned/crash_2018_cleaned.pkl")